In [ ]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
for dir in os.listdir('./data'):
    dir_path = os.path.join('./data', dir)
    if os.path.isdir(dir_path):
        num_files = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
        print(f"{dir} - {num_files} files")

In [ ]:
import os
import json

# 1. Đọc toàn bộ các mục từ JSON vào danh sách và xây dựng set + dict index
with open("symptom disease.json", "r", encoding="utf-8") as f:
    symptom_list = json.load(f)

# Tập các tên bệnh để kiểm tra nhanh
disease_names = { item['disease_name'] for item in symptom_list }

# Dict mapping từ disease_name -> vị trí (index) trong list
disease_index = {
    item['disease_name']: idx
    for idx, item in enumerate(symptom_list)
}

# 2. Lấy danh sách các thư mục con trong ./data
dirs = [
    d for d in os.listdir('./data')
    if os.path.isdir(os.path.join('./data', d))
]

# 3. Với mỗi dir, kiểm tra xem có trong disease_names không, và in cả vị trí
for d in dirs:
    if d in disease_names:
        idx = disease_index[d]
        print(f"[MATCH]    {d}  (position in JSON: {idx})  ✔️")
    else:
        print(f"[NO MATCH] {d}  ❌")


In [ ]:
import os
import matplotlib.pyplot as plt

# Tính số file trong mỗi thư mục
folder_counts = []
for dir in os.listdir('./data'):
    dir_path = os.path.join('./data', dir)
    if os.path.isdir(dir_path):
        num_files = len([
            f for f in os.listdir(dir_path)
            if os.path.isfile(os.path.join(dir_path, f))
        ])
        folder_counts.append((dir, num_files))

# Sắp xếp từ nhiều đến ít
folder_counts.sort(key=lambda x: x[1], reverse=True)

# Chuẩn bị dữ liệu
labels = [item[0] for item in folder_counts]
counts = [item[1] for item in folder_counts]

# Vẽ biểu đồ cột và in số
plt.figure(figsize=(12, 6))
bars = plt.bar(labels, counts)
plt.xlabel('Folder Name')
plt.ylabel('Number of Files')
plt.title('Số lượng file trong từng thư mục (từ nhiều đến ít)')
plt.xticks(rotation=45, ha='right')

# Ghi số lượng file lên đầu mỗi cột
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        height,
        str(int(height)),
        ha='center',
        va='bottom'
    )

plt.tight_layout()
plt.show()


In [ ]:
folder_dir = './data'
dataset = 'data_train_test'
if not os.path.exists(dataset):
    os.mkdir(dataset)
train_dir = './data_train_test/train_dir'
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
val_dir = './data_train_test/val_dir'
if not os.path.exists(val_dir):
    os.mkdir(val_dir)
all_images = []
all_labels = []
for dir in os.listdir(folder_dir):
    filepath = os.path.join(folder_dir, dir)
    
    train_path = os.path.join(train_dir, dir)
    if not os.path.exists(train_path):
        os.mkdir(train_path)
    val_path = os.path.join(val_dir, dir)
    if not os.path.exists(val_path):
        os.mkdir(val_path)
    for filename in os.listdir(filepath):
        name = os.path.join(dir, filename)
        all_images.append(name)
        all_labels.append(dir)


df_data = pd.DataFrame(
    {
        'image': all_images,
        'label': all_labels
    }
)
df_data


In [ ]:
y = df_data['label']
y
_, df_val = train_test_split(df_data, test_size=0.1, random_state=101, stratify=y)
df_val

In [ ]:
def identify_val_rows(x):
    
    val_list = list(df_val['image'])
    
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

df_data['train_or_val'] = df_data['image']
df_data['train_or_val'] = df_data['train_or_val'].apply(identify_val_rows)
df_train = df_data[df_data['train_or_val'] == 'train']
print(len(df_train))
print(len(df_val))

In [ ]:
df_train['label'].value_counts()

In [ ]:
df_val['label'].value_counts()

In [ ]:
df_data.set_index('image', inplace=True)

root = './data'
train_dir = './data_train_test/train_dir'
val_dir = './data_train_test/val_dir'

train_list = list(df_train['image'])
val_list = list(df_val['image'])

print(train_list)
print(val_list)

for image in tqdm(iterable=train_list, desc="train_list"):
    dir, name = image.split('\\')
    src = os.path.join(root, dir, name)
    dst = os.path.join(train_dir, dir, name)
    shutil.copyfile(src, dst)

for image in tqdm(iterable = val_list, desc="validation_list"):
    dir, name = image.split('\\')
    src = os.path.join(root, dir, name)
    dst = os.path.join(val_dir, dir, name)
    shutil.copyfile(src, dst)
